In [56]:
# Trim the audio so that we have 3 sec to train AI

from pydub import AudioSegment 

audio_file = AudioSegment.from_wav("ica.wav") 

first_3_seconds = audio_file[:10 * 385] 

first_3_seconds.export("trimmed_audio.wav", format="wav") 

<_io.BufferedRandom name='trimmed_audio.wav'>

In [57]:
# Print transcription from audio

from openai import OpenAI
client = OpenAI()

audio_file= open("ica.wav", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
transcription.text

'For my last holiday, I did a trip to Poland and a trip to Scotland. So, I started with a trip to Scotland to visit my friend for his wedding. It was a guy called Adam, and he was pretty fun as well. And we moved back to Poland with his girlfriend, who then very quickly became his fiancée. And in Poland...'

In [58]:
# trim string so that first 3 seconds are cut out

def remove_first_n_words(text, n):
    # Split the string into words
    words = text.split()
    
    # Check if there are at least n words
    if len(words) < n:
        return ""  # or return the original text if you prefer
    
    # Skip the first n words and rejoin the rest
    return ' '.join(words[n:])

transcription_text = remove_first_n_words(transcription.text, 9)
transcription_text

'Poland and a trip to Scotland. So, I started with a trip to Scotland to visit my friend for his wedding. It was a guy called Adam, and he was pretty fun as well. And we moved back to Poland with his girlfriend, who then very quickly became his fiancée. And in Poland...'

In [59]:
# Add new voice to API's provider

import requests

url = "https://api.play.ht/api/v2/cloned-voices/instant"

files = { "sample_file": ("trimmed_audio.wav.wav", open("trimmed_audio.wav", "rb"), "audio/wav") }
payload = { "voice_name": "Bob" }
headers = {
    "accept": "application/json",
    "AUTHORIZATION": "94fb497b6c6b4f478ce77c6d072dbe5e",
    "X-USER-ID": "QitPERiad2ViuQlOxRVEzYs15Zf2"
}

response = requests.post(url, data=payload, files=files, headers=headers)

print(response.text)

{"error_message":"You have reached the maximum number of cloned voices of (1).","error_id":"VOICE_CLONING_PACKAGE_LIMIT_REACHED"}


In [64]:
# Create a new file with the AI voice 
# Btw it startes after like 3 sec but later on we will combine those 2 so that it sounds good

# import the playht SDK
from pyht import Client, TTSOptions, Format

# Initialize PlayHT API with your credentials
client = Client("QitPERiad2ViuQlOxRVEzYs15Zf2", "94fb497b6c6b4f478ce77c6d072dbe5e")

# configure your stream
options = TTSOptions(
    # this voice id can be one of our prebuilt voices or your own voice clone id, refer to the`listVoices()` method for a list of supported voices.
    voice="s3://voice-cloning-zero-shot/17e3cb8c-da47-4924-b525-80d0b8428af7/bob/manifest.json",

    # you can pass any value between 8000 and 48000, 24000 is default
    sample_rate=44_100,
  
    # the generated audio encoding, supports 'raw' | 'mp3' | 'wav' | 'ogg' | 'flac' | 'mulaw'
    format=Format.FORMAT_WAV,

    # playback rate of generated speech
    speed=0.85,
)

# start streaming!
text = transcription_text

with open('output_audio.wav', 'wb') as audio_file:
    # Start streaming and write chunks to the file
    for chunk in client.tts(text=text, voice_engine="PlayHT2.0-turbo", options=options):
        audio_file.write(chunk)

In [65]:
from pydub import AudioSegment

def combine_wav_files_diff_params(file1_path, file2_path, output_path):
    # Load the first file
    sound1 = AudioSegment.from_wav(file1_path)
    
    # Load the second file
    sound2 = AudioSegment.from_wav(file2_path)
    
    # Combine the two sounds
    combined_sounds = sound1 + sound2
    
    # Export the combined sound to a new file
    combined_sounds.export(output_path, format='wav')

# Example usage
combine_wav_files_diff_params('trimmed_audio.wav', 'output_audio.wav', 'final_one.wav')
